<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Gestures_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd